In [21]:
import pandas as pd
import os
import re
from sqlalchemy import create_engine, text, inspect

# --- 1. Import Credentials ---
try:
    from directory import DB_USER, DB_PASS, DB_HOST, DB_PORT, DB_NAME
except ImportError:
    print("FATAL ERROR: Could not import database credentials.")
    raise

# --- 2. Connect ---
db_url = f'postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
try:
    engine = create_engine(db_url)
    print(f"Connected to: '{DB_NAME}'")
except Exception as e:
    print(f"Connection failed: {e}")
    raise

# --- 3. Find CSVs ---
clean_csvs = [f for f in os.listdir() if f.endswith('.csv')]
print(f"Found {len(clean_csvs)} CSV files.")

# --- 4. Smart Upload (Auto-Fix Columns) ---
inspector = inspect(engine)

for csv_file in clean_csvs:
    try:
        # A. Prepare Table Name (Max 63 chars)
        table_name_base = os.path.splitext(csv_file)[0].lower()
        table_name = re.sub(r'[^a-zA-Z0-9_]', '_', table_name_base)
        if len(table_name) > 63:
            table_name = table_name[:63]
            if table_name.endswith('_'): table_name = table_name[:-1]

        print(f"Processing: {table_name}...")

        # B. Read CSV
        df = pd.read_csv(csv_file)

        # C. Check for Missing Columns in DB (The Fix for "Assister" error)
        if inspector.has_table(table_name):
            # Get existing DB columns
            existing_columns = [c['name'] for c in inspector.get_columns(table_name)]

            # Find columns in CSV that are NOT in DB
            new_columns = [c for c in df.columns if c not in existing_columns]

            if new_columns:
                print(f"  -> Found new columns in CSV: {new_columns}")
                print(f"  -> Adding them to database table '{table_name}'...")

                with engine.connect() as conn:
                    for col in new_columns:
                        # We add new columns as TEXT to be safe and flexible
                        # If you need specific types (int/float), complex logic is needed,
                        # but TEXT works for almost everything in a pinch.
                        alter_query = text(f'ALTER TABLE "{table_name}" ADD COLUMN "{col}" TEXT')
                        conn.execute(alter_query)
                    conn.commit()
                    print("  -> Schema updated successfully.")

        # D. Truncate and Append
        with engine.connect() as conn:
            # 1. Wipe the table (Truncate)
            # We try/except this in case the table implies doesn't exist yet
            try:
                conn.execute(text(f'TRUNCATE TABLE "{table_name}"'))
                conn.commit()
            except Exception:
                conn.rollback() # Ignore if table doesn't exist yet

        # 2. Upload data (Append)
        df.to_sql(table_name, engine, if_exists='append', index=False)

    except Exception as e:
        print(f"  !! CRITICAL ERROR on {csv_file}: {e}")

print("\nRecovery complete. All tables should now have data.")
engine.dispose()

Connected to: 'soccer_data'
Found 110 CSV files.
Processing: 2024_25_spanish_copa_del_rey__semifinals_fixtures...
Processing: 2024_25_spanish_copa_del_rey__qualifying_round_fixtures...
Processing: 2024_25_german_bundesliga_standings...
Processing: 2024_25_english_fa_cup__keyevents_data...
Processing: 2024_25_coppa_italia__round_of_16_fixtures...
Processing: 2024_25_coupe_de_france__quarterfinals_fixtures...
Processing: 2024_25_spanish_copa_del_rey__first_round_fixtures...
Processing: 2024_25_uefa_europa_league__knockout_round_playoffs_fixtures...
Processing: 2024_25_english_fa_cup__semifinals_fixtures...
Processing: 2024_25_english_fa_cup__final_fixtures...
Processing: 2024_25_english_fa_cup__plays_data...
Processing: 2024_25_coppa_italia__quarterfinals_fixtures...
Processing: 2024_25_german_cup___plays_data...
Processing: 2024_25_uefa_champions_league__plays_data...
Processing: 2024_25_ligue_1_plays_data...
Processing: 2024_25_uefa_conference_league__final_fixtures...
Processing: 2024